In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('adult.data', header=None)

In [3]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']

In [4]:
df.columns = column_names

In [5]:
import numpy as np

In [6]:
df.replace('?', np.nan).dropna()

age          workclass  fnlwgt    education  education-num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spouse     Exec-managerial         Husband   White   
2                 Divorced   Handlers-cleaners   Not-in-family   White   
3       Married-civ-spouse   Handlers-cleaners         Husband   Black   
4       Married-civ-spouse      Prof-specialty            Wife   Black   
...                    ...                 ...             ...     ...   
32556   Married-civ-spouse        Tech-support            Wife   White   
32557   Married-civ-spouse   Machine-op-inspct         Husband   White   
32558              Widowed        Adm-clerical       Unmarried   White   
32559        Never-married        Adm-clerical       Own-child   White   
32560   Married-civ-spouse     Exec-managerial            Wife   White   

           sex  capital-gain  capital-loss  hours-per-week  native-country  \
0         Male          2174             0              40   United-States   
1         Male             0             0              13   United-States   
2         Male             0             0              40   United-States   
3         Male             0             0              40   United-States   
4       Female             0             0              40            Cuba   
...        ...           ...           ...             ...             ...   
32556   Female             0             0              38   United-States   
32557     Male             0             0              40   United-States   
32558   Female             0             0              40   United-States   
32559     Male             0             0              20   United-States   
32560   Female         15024             0              40   United-States   

        income  
0       income  
1        <=50K  
2        <=50K  
3        <=50K  
4        <=50K  
...        ...  
32556    <=50K  
32557     >50K  
32558    <=50K  
32559    <=50K  
32560     >50K  

[32561 rows x 15 columns]

In [7]:
X = df.drop('income', axis=1)

In [8]:
Y = df['income']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_trian, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1293)

In [11]:
numeric_columns = X.select_dtypes(include=["int64", "float64"]).columns

In [12]:
catagorical_columns = X.select_dtypes(include=['object']).columns

In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [14]:
preprocessing = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), catagorical_columns)
])
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [15]:
clf = Pipeline([
    ('preprocessor', preprocessing),
    ('classifier', RandomForestClassifier(n_estimators=100,random_state=1293))])

In [16]:
clf.fit(X_trian, Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=1293))])

In [17]:
y_prediction = clf.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

In [18]:
print(f"Acurracy: {accuracy_score(Y_test, y_prediction):.4f}")

Acurracy: 0.8544


In [19]:
print(classification_report(Y_test, y_prediction))

              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91      4933
        >50K       0.74      0.61      0.67      1580

    accuracy                           0.85      6513
   macro avg       0.81      0.77      0.79      6513
weighted avg       0.85      0.85      0.85      6513



In [20]:
import joblib

In [22]:
joblib.dump(clf, 'model.joblib')

['model.joblib']

In [23]:
loaded_model = joblib.load('model.joblib')

In [30]:
def predictions(input_dataset):
    input_DataFrame = pd.DataFrame([input_dataset], columns=X.columns)
    prediction = loaded_model.predict(input_DataFrame)
    return prediction[0]

Predicted income =  <=50K
